In [2]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from lxml import html
from urllib.error import HTTPError
import pandas as pd

In [3]:
def retrieve_unique_from_element(soup_object, class_name, child=None, child_class=None):
    if child_class != None:
        element_value = soup_object.find('div', {'class': class_name}).find(child, {'class': child_class}).text
    elif child != None:
        element_value = soup_object.find('div', {'class': class_name}).find(child).text
    else:
        element_value = soup_object.find('div', {'class': class_name}).text
    return element_value

def retrieve_bunch_from_element(soup_object, class_name, child=None, child_class=None):
    member_list = soup_object.find('div', {'class': class_name}) 
    element_value_list = []
    for child in member_list.findChildren():
        if child.text[0].isdigit():
            element_value_list.append(int(child.text.split(" ", 1)[0]))
    return element_value_list

class_list = [['mb-2', 'h1'],['item-year'],['address'],['type-name'],['project-price', 'span'],
              ['cost budget', 'span', 'number'], ['cost municipal', 'span', 'number'],
              ['cost citizens', 'span', 'number'], ['cost business', 'span', 'number']] 

def get_data(project_id, html_file, class_list):
    soup_html = BeautifulSoup(html_file)
    project_features = [project_id]
    for i in class_list:
        if len(i) == 3:
            project_features.append(retrieve_unique_from_element(soup_html, i[0], i[1], i[2]))
        elif len(i) == 2:
            project_features.append(retrieve_unique_from_element(soup_html, i[0], i[1]))
        else:
            project_features.append(retrieve_unique_from_element(soup_html, i[0]))
    project_features.extend(retrieve_bunch_from_element(soup_html, 'members-count')) 
    return project_features

def parse_data():
    project_id = 1
    ib_dataset = pd.DataFrame()
    while project_id < 100000: # projects are de facto found up to the number 70000
        try: 
            response = urlopen("https://budget4me.ru/projects/view-project/?view="+str(project_id)) 
            #print(project_id)
            if project_id % 100 == 0:
                print(f"Processing {project_id}...")
            html_project_page = response.read().decode('utf-8')
            dataset_row = pd.DataFrame(get_data(project_id, html_project_page, class_list)).T
            if len(dataset_row.columns) == 13:
                print(f'{project_id} - BINGO')
                ib_dataset = ib_dataset.append(dataset_row)
        except AttributeError:
            print(f'{project_id}: AttributeError') 
        except HTTPError:
            print(f'{project_id}: Error 404')
        finally:
            project_id += 1
    ib_dataset.columns = ['project_id','project_description','year','address','category','project_cost',
                          'fund_region','fund_municipality','fund_citizens','fund_business',
                          'initiative_group','assembly_participants','beneficiaries']
    return ib_dataset

In [6]:
output_data = parse_data()
#output_data.to_csv('init_budg_projects.csv', index=False) 